In [1]:
import os
import pandas as pd
from keras.preprocessing.text import Tokenizer

In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/6-class-bert-preprocessed-dataset/Bert_Preprocessed_Dataset.xlsx


In [3]:
pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 KB 5.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
df=pd.read_excel('/kaggle/input/6-class-bert-preprocessed-dataset/Bert_Preprocessed_Dataset.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6405 entries, 0 to 6404
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6405 non-null   int64 
 1   ID          6404 non-null   object
 2   Text        6404 non-null   object
 3   Sentiment   6405 non-null   object
 4   Label       6405 non-null   int64 
 5   Token       6405 non-null   object
 6   Token_ID    6405 non-null   object
dtypes: int64(2), object(5)
memory usage: 350.4+ KB


In [5]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Concatenate, Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Flatten
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Convolution1D
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.layers import Add, BatchNormalization, Activation, Dropout

In [6]:
MAX_FEATURES = 10000
EMBED_SIZE = 128
tokenizer = Tokenizer(num_words=MAX_FEATURES)
df['Text']=df['Text'].astype(str)
tokenizer.fit_on_texts(df['Text'])



In [7]:
train=df[:int(.8*len(df))]
test=df[int(.8*len(df)):int(.9*len(df))]
validation=df[int(.9*len(df)):]
x_train=train['Text'].astype(str)
y_train=train['Label']
x_validation=validation['Text'].astype(str)
y_validation=validation['Label']
x_test=test['Text'].astype(str)
y_test=test['Label']

In [8]:

list_tokenized_train = tokenizer.texts_to_sequences(x_train)
list_tokenized_val = tokenizer.texts_to_sequences(x_validation)  
list_tokenized_test = tokenizer.texts_to_sequences(x_test)  
RNN_CELL_SIZE = 100
  
MAX_LEN = 70  
X_train = pad_sequences(list_tokenized_train, maxlen=MAX_LEN)
X_valid = pad_sequences(list_tokenized_val, maxlen=MAX_LEN)
X_test = pad_sequences(list_tokenized_test, maxlen=MAX_LEN)

In [9]:
class Attention(tf.keras.Model):
    def __init__(self, units):
        super(Attention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
          
        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = tf.nn.tanh(
            self.W1(features) + self.W2(hidden_with_time_axis))
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(self.V(score), axis=1)
          
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

In [10]:
sequence_input = Input(shape=(MAX_LEN,), dtype="int32")
embedded_sequences = Embedding(MAX_FEATURES, EMBED_SIZE)(sequence_input)

2022-04-18 14:13:14.058388: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [11]:
#conv = Convolution1D(128,5,activation='relu')(embedded_sequences)
#pol = GlobalMaxPool1D()(conv)

lstm = Bidirectional(LSTM(64, return_sequences = True), name="bi_lstm_0")(embedded_sequences)

# Getting our LSTM outputs
(lstm, forward_h, forward_c, backward_h, backward_c) = Bidirectional(LSTM(64, return_sequences=True, return_state=True), name="bi_lstm_1")(lstm)

In [12]:
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
context_vector, attention_weights = Attention(20)(lstm, state_h)
dense1 = Dense(20, activation="relu")(context_vector)
dropout = Dropout(0.1)(dense1)
output = Dense(6, activation="softmax")(dropout)
  
model = keras.Model(inputs=sequence_input, outputs=output)

In [13]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer= keras.optimizers.Adam(learning_rate = 0.001),
              metrics=['accuracy'])

In [14]:
BATCH_SIZE = 64
EPOCHS = 20
history = model.fit(X_train,y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(X_valid, y_validation))

2022-04-18 14:13:15.661122: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20
81/81 [==============================] - 27s 237ms/step - loss: 1.6407 - accuracy: 0.3247 - val_loss: 1.5297 - val_accuracy: 0.3682
Epoch 2/20
81/81 [==============================] - 16s 202ms/step - loss: 1.4028 - accuracy: 0.4383 - val_loss: 1.4814 - val_accuracy: 0.3994
Epoch 3/20
81/81 [==============================] - 17s 210ms/step - loss: 1.1643 - accuracy: 0.5482 - val_loss: 1.5348 - val_accuracy: 0.4540
Epoch 4/20
81/81 [==============================] - 17s 204ms/step - loss: 0.8760 - accuracy: 0.6805 - val_loss: 1.7127 - val_accuracy: 0.4493
Epoch 5/20
81/81 [==============================] - 17s 213ms/step - loss: 0.6237 - accuracy: 0.7873 - val_loss: 1.9759 - val_accuracy: 0.4571
Epoch 6/20
81/81 [==============================] - 16s 202ms/step - loss: 0.4640 - accuracy: 0.8501 - val_loss: 2.1855 - val_accuracy: 0.4431
Epoch 7/20
81/81 [==============================] - 17s 211ms/step - loss: 0.3778 - accuracy: 0.8829 - val_loss: 2.3168 - val_accuracy: 0.4446

In [15]:
import numpy as np
from sklearn.metrics import classification_report

In [16]:
y_pred = np.argmax(model.predict(X_test),axis = -1)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.42      0.51      0.46       144
           1       0.18      0.18      0.18        66
           2       0.09      0.09      0.09        32
           3       0.70      0.57      0.63       187
           4       0.37      0.34      0.36       148
           5       0.21      0.27      0.23        63

    accuracy                           0.41       640
   macro avg       0.33      0.33      0.33       640
weighted avg       0.43      0.41      0.42       640

